# Install packages

In [8]:
!pip3 install imutils
!pip3 install opencv-python

!apt-get install graphviz

!pip3 install graphviz==0.5.2
!pip3 install pydot-ng==1.0.0

/bin/sh: apt-get: command not found
/bin/sh: pip3 install graphviz==0.5.2: command not found
/bin/sh: pip3 install pydot-ng==1.0.0: command not found


# Load dataset

In [2]:
from imutils import paths
from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader


dataset_path = './datasets/plantVillage-full-mohanty'

# Get the list of image names
image_paths = list(paths.list_images(dataset_path))
# Get unique classnames
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

# Preprocessors
aap = AspectAwarePreprocessor(32, 32)
itap = ImageToArrayPreprocessor()

# Add preprocessors to DataSetLoader class
sdl = SimpleDatasetLoader(preprocessors=[aap, itap])

(data, labels) = sdl.load(image_paths, verbose=500)
data = data.astype('float') / 255.0


Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

NameError: name 'imagePaths' is not defined

# Encode labels and split dataset

In [0]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
test_size=0.25, random_state=42)
∑
# Convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)


# Initialize AlexNet Model

In [0]:
from utils.models.alexnet import alexnet
from keras.utils import plot_model

model = alexnet()
model.summary()
plot_model(model, to_file="alex.png", show_shapes=True)

# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])


# Fit model

In [0]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),
              batch_size=32, epochs=100, verbose=1)

model.save('alexnet_model.h5')

# Generate classfication report

In [0]:
from sklearn.metrics import classification_report


# If one hot encoding is used
print(classification_report(test_y, model.predict(test_x),
                            target_names=classNames))


# Plot model

In [0]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])
plt.show()